In [ ]:
# !pip install jupylet

In [86]:
from jupylet.label import Label
from jupylet.app import App

In [ ]:
# hello = Label('hello, world', color='cyan', font_size=32, x=app.width, y=22)

In [88]:
#@app.run_me_every(1/60)
#def scroll(ct, dt):
#    hello.x -= dt * 48
#    if hello.right < 0:
#        hello.x = app.width

In [83]:
from enum import Enum
import numpy as np
import random

class Terrain(Enum):
    PLAIN = 1
    HILL = 2
    MOUNTAIN = 3
    MEADOW = 4
    FOREST = 5
    DESERT = 6
    COAST = 7
    OCEAN = 8
    BEACH = 9
    LAKE = 10

T = Terrain

adjs = {
    T.PLAIN: set([T.HILL, T.MEADOW, T.BEACH, T.DESERT]),
    T.HILL: set([T.MOUNTAIN, T.LAKE]),
    T.MOUNTAIN: set([T.HILL, T.LAKE]),
    T.MEADOW: set([T.FOREST, T.PLAIN]),
    T.FOREST: set([T.MEADOW]),
    T.DESERT: set([T.PLAIN, T.HILL]),
    T.COAST: set([T.OCEAN, T.BEACH]),
    T.OCEAN: set([T.COAST]),
    T.BEACH: set([T.COAST, T.BEACH, T.DESERT]),
    T.LAKE: set([T.MOUNTAIN, T.HILL])
}

for k, v in list(adjs.items()):
    adjs[k].add(k)
    for k2 in v:
        adjs[k2].add(k)

def next_cell(t):
    opts = np.vectorize(lambda o: 999 if len(o) == 1 else len(o), signature="()->()")(t)

    c = np.unravel_index(opts.argmin(), opts.shape)

    if len(t[c]) > 1:
        return c
    else:
        return None

def adjacent(t, n):
    for d in range(len(t.shape)):
        r = list(n)
        r[d] -= 1
        if r[d] >= 0:
            yield tuple(r)
        r = list(n)
        r[d] += 1
        if r[d] < t.shape[d]:
            yield tuple(r)

def gen_terrain(w, h):
    t = np.empty(shape=(w, h), dtype=np.object_)

    for x in range(w):
        for y in range(h):
            t[x][y] = set(Terrain)

    n = next_cell(t)

    while n is not None:
        opts = t[n]

        pick = random.choice(list(opts))

        t[n] = set([pick])

        constraints = adjs[pick]

        for a in adjacent(t, n):
            t[a] = t[a].intersection(constraints)

        n = next_cell(t)

    return t

In [95]:
t = gen_terrain(10, 10)

In [96]:
t

array([[{<Terrain.MOUNTAIN: 3>}, {<Terrain.MOUNTAIN: 3>},
        {<Terrain.MOUNTAIN: 3>}, {<Terrain.HILL: 2>},
        {<Terrain.HILL: 2>}, {<Terrain.LAKE: 10>}, {<Terrain.LAKE: 10>},
        {<Terrain.HILL: 2>}, {<Terrain.HILL: 2>},
        {<Terrain.MOUNTAIN: 3>}],
       [{<Terrain.LAKE: 10>}, {<Terrain.MOUNTAIN: 3>},
        {<Terrain.MOUNTAIN: 3>}, {<Terrain.MOUNTAIN: 3>},
        {<Terrain.MOUNTAIN: 3>}, {<Terrain.MOUNTAIN: 3>},
        {<Terrain.HILL: 2>}, {<Terrain.HILL: 2>}, {<Terrain.HILL: 2>},
        {<Terrain.LAKE: 10>}],
       [{<Terrain.LAKE: 10>}, {<Terrain.MOUNTAIN: 3>},
        {<Terrain.LAKE: 10>}, {<Terrain.HILL: 2>},
        {<Terrain.MOUNTAIN: 3>}, {<Terrain.MOUNTAIN: 3>},
        {<Terrain.LAKE: 10>}, {<Terrain.LAKE: 10>},
        {<Terrain.MOUNTAIN: 3>}, {<Terrain.LAKE: 10>}],
       [{<Terrain.HILL: 2>}, {<Terrain.MOUNTAIN: 3>},
        {<Terrain.MOUNTAIN: 3>}, {<Terrain.MOUNTAIN: 3>},
        {<Terrain.MOUNTAIN: 3>}, {<Terrain.MOUNTAIN: 3>},
        {<Terrai

In [103]:
app = App(width=640, height=640)

labels = np.empty(shape=t.shape, dtype=np.object_)

for x in range(labels.shape[0]):
    for y in range(labels.shape[1]):
        txt = next(iter(t[x, y])).name[0:3]
        labels[x, y] = Label(txt, x=(x+1)*50, y=(y+1)*50)

@app.event
def render(ct, dt):
    app.window.clear()
    for x in range(labels.shape[0]):
        for y in range(labels.shape[1]):
            labels[x, y].draw()


app.run()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…